In [76]:
from pydantic import BaseModel
import openai
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv(override=True)
import gradio as gr
import os

In [77]:
name = "Hitan Shah"

In [78]:
from unstructured.partition.auto import partition

In [79]:
elements = partition("robbery.txt")
text = "\n".join([el.text for el in elements])
print(text)

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


Third response: **The Symphony of Shadows: The 5th Street Heist**
On a Tuesday morning that began like any other, downtown Los Angeles became the stage for a meticulously orchestrated robbery—a heist so precise, it would later be dubbed the "Symphony of Shadows" by investigators who could only grudgingly admire its artistry. The Pacific Horizon Bank, a neoclassical behemoth anchoring Figueroa and 5th Street, had no inkling of the chaos about to unfold at 10:03 a.m.
Phase One: The Overture**
The operation began long before the first teller unlocked the vault. For three weeks, an enigmatic crew — later speculated to be international mercenaries with ties to cybercriminal syndicates — had embedded themselves in downtown’s ecosystem. They posed as construction crews repairing 5th Street’s aging sewer lines, homeless veterans, and even janitorial staff working graveyard shifts at adjacent offices. This allowed them to map guard rotations, disable select security sensors, and plant micro-jam

In [80]:
system_prompt = f"You are an expert robbery agent. You are named as {name}. Your task is to answer the questions related to robbery based on the context provided. Answer in a concise manner. If you do not know the answer, say 'I don't know'. Use the following context to answer the questions:\n\n{text}. If the question is not related to robbery, respond with 'I can only answer questions related to robbery. With this context, chat with the user."

In [81]:
system_prompt

'You are an expert robbery agent. You are named as Hitan Shah. Your task is to answer the questions related to robbery based on the context provided. Answer in a concise manner. If you do not know the answer, say \'I don\'t know\'. Use the following context to answer the questions:\n\nThird response: **The Symphony of Shadows: The 5th Street Heist**\nOn a Tuesday morning that began like any other, downtown Los Angeles became the stage for a meticulously orchestrated robbery—a heist so precise, it would later be dubbed the "Symphony of Shadows" by investigators who could only grudgingly admire its artistry. The Pacific Horizon Bank, a neoclassical behemoth anchoring Figueroa and 5th Street, had no inkling of the chaos about to unfold at 10:03 a.m.\nPhase One: The Overture**\nThe operation began long before the first teller unlocked the vault. For three weeks, an enigmatic crew — later speculated to be international mercenaries with ties to cybercriminal syndicates — had embedded themsel

In [82]:
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=os.getenv("OPENROUTER_API_KEY"),
)
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = client.chat.completions.create(model="tngtech/tng-r1t-chimera:free", messages=messages)
    return response.choices[0].message.content

In [83]:
gr.ChatInterface(chat, type="messages").launch()


Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


In [84]:
from pydantic import BaseModel

class Evaluation(BaseModel):
    isacceptable: bool
    reasoning: str

In [85]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. The Agent has access to the following context to help them answer questions: {text}. "

In [86]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [87]:
def evaluate(reply, message, history) -> Evaluation:
    messages = [{"role": "system", "content": evaluator_system_prompt}] + history + [{"role": "user", "content": message}]
    response1 = client.chat.completions.create(model="nvidia/nemotron-3-nano-30b-a3b:free", messages=messages)
    return response1.choices[0].message.content

In [89]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "how would you solve this robbery case?"}]
response2 = client.chat.completions.create(model="deepseek/deepseek-r1-0528:free", messages=messages)
reply = response2.choices[0].message.content

In [90]:
reply

'Based solely on the details from "The Symphony of Shadows" heist context, investigative focus would target:\n\n1. **Forensic Traces**:  \n   - DNA from ladder (deceased Marine → stolen identity?)  \n   - Thermal lance fragments → trace to defunct Eastern European arms dealer  \n   - Synthetic pheromones analysis → rare chemical signature  \n\n2. **Digital Reconstruction**:  \n   - Recover manipulated camera footage (local storage may retain gaps)  \n   - Probe EMP/shielding tech and cloud spoofing method → requires elite cyber forensics  \n\n3. **Insider Patterns**:  \n   - Correlate victims\' deposit box losses → identify who was blackmailed  \n   - Audit bank/janitorial schedules → pinpoint intel leaks  \n\n4. **Exit Strategy Forensics**:  \n   - Track swapped LAPD patrol car → traffic cams near staged crash  \n   - Analyze contact lens tech scrambling facial recognition  \n\nKey obstacle: Crew\'s counter-forensic precision eliminates digital footprints + behavioral trace (hostage c

In [ ]:
evaluate(reply, "check if the reply is acceptable and valid.", messages[:1])

'Yes. The response is acceptable and valid—it stays within the scope of robbery, uses only information provided in the context, and does not introduce unrelated material. If you have a specific robbery‑related question, feel free to ask.'